## Algoritmo de Regresiones
### Lineales y No Lineales

Alumno: Leonardo Neyra Ocaña

UPAO 2018-10

In [2]:
#importar paquetes
import numpy as np
import random as rn
import pandas as pd
import math as mt

In [3]:
#Funcion de Creacion del dataset
def crearDataSet():
    x = [14,7,13,12,16,14,18,13,12,16,13]
    y = [16,12,13,14,15,12,16,11,13,18,17]
    df = pd.DataFrame({'X': x, 'Y': y})
    return df

In [8]:
#Generar dataset
datos = crearDataSet()
datos

,X,Y
0,14,16
1,7,12
2,13,13
3,12,14
4,16,15
5,14,12
6,18,16
7,13,11
8,12,13
9,16,18


In [9]:
#Funcion para calcular matriz
#Source: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
#Source: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html
def calcularMatriz(dt):
    xy = (dt.loc[:,'X'])*(dt.loc[:,'Y'])
    x2 = (dt.loc[:,'X'])*(dt.loc[:,'X'])
    y2 = (dt.loc[:,'Y'])*(dt.loc[:,'Y'])
    #Agregar columnas
    dt['XY'] = xy
    dt['X2'] = x2
    dt['Y2'] = y2
    sx = sum(dt.X)
    sy = sum(dt.Y)
    sxy = sum(dt.XY)
    sx2 = sum(dt.X2)
    sy2 = sum(dt.Y2)
    #Constructing DataFrame from numpy ndarray
    #Agregar una fila con los totalizados
    temp = pd.DataFrame([[sx, sy, sxy, sx2, sy2]], columns=['X','Y','XY','X2','Y2'])
    dt = dt.append(temp, ignore_index = True)
    return dt

In [20]:
#Calcular matriz
matriz = calcularMatriz(datos)
matriz

,X,Y,XY,X2,Y2
0,14,16,224,196,256
1,7,12,84,49,144
2,13,13,169,169,169
3,12,14,168,144,196
4,16,15,240,256,225
5,14,12,168,196,144
6,18,16,288,324,256
7,13,11,143,169,121
8,12,13,156,144,169
9,16,18,288,256,324


In [61]:
#Funcion para calcular R(Coeficiente de correlacion)
def calcularR(dt):
    promx = dt.loc[11:,'X']/11
    promy = dt.loc[11:,'Y']/11
    promx2 = promx*promx
    promy2 = promy*promy
    covxy = (dt.loc[11:,'XY']/11)-(promx*promy)
    desvx = mt.sqrt((dt.loc[11:,'X2']/11)-promx2)
    desvy = mt.sqrt((dt.loc[11:,'Y2']/11)-promy2)
    r = (covxy)/(desvx*desvy)
    dt = pd.DataFrame([[promx, promy, covxy, desvx, desvy, r]], 
                      columns=['PromX','PromY','CovXY','DesvX','DesvY','R'])
    return dt

In [62]:
#Calcular R
R = calcularR(matriz)
R

,PromX,PromY,CovXY,DesvX,DesvY,R
0,"11 13.454545 Name: X, dtype: float64","11 14.272727 Name: Y, dtype: float64",11 3.330579 dtype: float64,2.70903,2.178027,11 0.564472 dtype: float64


In [63]:
#Funcion para obtener un nuevo X
def nuevoY(dt, x):
    newy = ((dt.CovXY/(dt.DesvX*dt.DesvX))*(x-dt.PromX))+dt.PromY
    return newy

#Funcion para obtener un nuevo X
def nuevoX(dt, y):
    newx = ((dt.CovXY/(dt.DesvY*dt.DesvY))*(y-dt.PromY))+dt.PromX
    return newx

In [64]:
#Calcular nuevo X
nuevoX(R,28)

0    11    23.092334
dtype: float64
dtype: object

In [65]:
#Calcular nuevo Y
nuevoY(R,28)

0    11    20.873874
dtype: float64
dtype: object

#### Ahora vamos a implementar la funcion para predecir por medio del metodo de minimos cuadrados para regresiones no lineales (en este caso se usará una regresion cuadrática)

In [68]:
#Funciona para minimos cuadrados
def calcularMinCuadrados(dt, newx):
    xy = (dt.loc[:,'X'])*(dt.loc[:,'Y'])
    x2 = (dt.loc[:,'X'])*(dt.loc[:,'X'])
    x3 = (dt.loc[:,'X'])*(dt.loc[:,'X'])*(dt.loc[:,'X'])
    x4 = (dt.loc[:,'X'])*(dt.loc[:,'X'])*(dt.loc[:,'X'])*(dt.loc[:,'X'])
    x2y = (dt.loc[:,'X'])*(dt.loc[:,'X'])*(dt.loc[:,'Y'])
    #Agregar columnas
    dt['XY'] = xy
    dt['X2'] = x2
    dt['X3'] = x3
    dt['X4'] = x4
    dt['X2Y'] = x2y
    #Sumatorias
    sx = sum(dt.X)
    sy = sum(dt.Y)
    sx2 = sum(dt.X2)
    sx3 = sum(dt.X3)
    sx4 = sum(dt.X4)
    sx2y = sum(dt.X2Y)
    sxy = sum(dt.XY)
    #Definimos otra funcion
    def calcularDeterminante(a1,b1,c1,a2,b2,c2,a3,b3,c3):
        determ = (a1*b2*c3 + b1 * c2*a3 + a2 * b3*c1) - (c1*b2*a3 + b1 * a2*c3 + c2 * b3*a1)
        return determ
    #Continuamos calculando las determinantes
    #El numero 11 representa la longitud del dataframe
    Determinante = calcularDeterminante(11, sx, sx2, sx, sx2, sx3, sx2, sx3, sx4)
    DeterminanteX = calcularDeterminante(sy, sx, sx2, sxy, sx2, sx3, sx2y, sx3, sx4)
    DeterminanteY = calcularDeterminante(11, sy, sx2, sx, sxy, sx3, sx2, sx2y, sx4)
    DeterminanteZ = calcularDeterminante(11, sx, sy, sx, sx2, sxy, sx2, sx3, sx2y)
    #Calculamos los coeficientes de la funcion
    #La funcion tiene la forma: y = a0 + a1*x + a2*(x^2)
    a0 = DeterminanteX / Determinante
    a1 = DeterminanteY / Determinante
    a2 = DeterminanteZ / Determinante
    #Ahora que ya tenemos los coeficientes determinamos la recta en funcion de X
    rectaY = a0 + (a1*newx) + (a2*(newx*newx))
    #Retornamos el valor de Y en funcion de X
    return rectaY

In [69]:
#Calcular nuevo X con metodo de minimos cuadrados
calcularMinCuadrados(datos, 28)

26.39596989560573